# Cryptography CTF Basics
by Daniel Mancia

In [1]:
# Useful python packages
! pip install --user gmpy2
! pip install --user pycryptodome

# I also highly recommend downloading SAGE (http://www.sagemath.org)

## Base Representation of Numbers

In [2]:
n = 11111
print(bin(n)) # Base 2  (binary)
print(oct(n)) # Base 8  (octal)
print(hex(n)) # Base 16 (hex)

n = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
print(int(n, 36))


0b10101101100111
0o25547
0x2b67
8337503854730415241050377135811259267835


## Data Representation

Data can be represented in many ways. The most common way is through bytes and hexadecimal.

Releveant docs:  
https://docs.python.org/3/library/stdtypes.html#bytes  
https://docs.python.org/3/library/stdtypes.html#str.encode  
https://docs.python.org/3/library/stdtypes.html#bytes.decode

In [12]:
s = 'OK Boomer!'
print(s)

# Use encode to get the byte representation of a string
s_bytes = s.encode()
print(s_bytes)

# You could also just add b in front of the string (only works with ascii literals)
t_bytes = b'yeet'
print(t_bytes)

# Decode it back to a string with .decode()
print(t_bytes.decode())

# bytes to string
a = [0x62, 0x79, 0x74, 0x65, 0x73]
print(bytes(a))

OK Boomer!
b'OK Boomer!'
b'yeet'
yeet
b'bytes'


In [23]:
# Python3 strings support unicode characters!
# https://docs.python.org/3/howto/unicode.html

boom = '🤯'
print(boom)
print(boom.encode())

test = b'\xf0\x9f\xa4\xae'
print(test)
print(test.decode())

🤯
b'\xf0\x9f\xa4\xaf'
b'\xf0\x9f\xa4\xae'
🤮


In [13]:
# Manually get hex representation
print(s_bytes)
s_hex = ''
for byte in s_bytes:
    s_hex += str(hex(byte)[2:]) + ' '
print(s_hex)

# Or just do this
print(s_bytes.hex())

# Inverse operation is fromhex
print(bytes.fromhex(s_bytes.hex()))
print(bytes.fromhex(s_hex))

b'OK Boomer!'
4f 4b 20 42 6f 6f 6d 65 72 21 
4f4b20426f6f6d657221
b'OK Boomer!'
b'OK Boomer!'


## Base64 Encoding

A common encoding scheme used in CTFs is Base64 encoding. 

From wikipedia: "Base64 is a group of binary-to-text encoding schemes that represent binary data in an ASCII string format by translating it into a radix-64 representation."

In [16]:
import base64

print(s_bytes)
encoded = base64.b64encode(s_bytes)
print(encoded.decode() + '\n')

decoded = base64.b64decode(encoded)
print(decoded)

b'OK Boomer!'
T0sgQm9vbWVyIQ==

b'OK Boomer!'


## Single Byte Xor

Xor is associative: $(a \oplus b) \oplus c = a \oplus (b \oplus c)$  
More importantly $a \oplus a = 0$  

Therefore if we know that every single byte of the ciphertext was xored with a single character we can use xor properties:
$plaintext_i \oplus key = ciphertext_i \iff plaintext_i \oplus key \oplus key = ciphertext_i \oplus key \iff plaintext_i = ciphertext_i \oplus key$

This means we can brute force all 255 characters to find the key

In [26]:
# Suppose we know that the flag is encrypted by a single byte
garbage = b'\x1c\x1d\x0f\x05\x08\x0e\x12\x0f\x08\x02\x0c\x0f\x05\x08\x0e\x14'
print(garbage.decode('ascii'))
candidates = []
for c in range(0, 256):
    candidate = []
    for x in garbage:
        candidate.append(c ^ x)
    candidates.append(bytes(candidate))
print(candidates)


[b'\x1c\x1d\x0f\x05\x08\x0e\x12\x0f\x08\x02\x0c\x0f\x05\x08\x0e\x14', b'\x1d\x1c\x0e\x04\t\x0f\x13\x0e\t\x03\r\x0e\x04\t\x0f\x15', b'\x1e\x1f\r\x07\n\x0c\x10\r\n\x00\x0e\r\x07\n\x0c\x16', b'\x1f\x1e\x0c\x06\x0b\r\x11\x0c\x0b\x01\x0f\x0c\x06\x0b\r\x17', b'\x18\x19\x0b\x01\x0c\n\x16\x0b\x0c\x06\x08\x0b\x01\x0c\n\x10', b'\x19\x18\n\x00\r\x0b\x17\n\r\x07\t\n\x00\r\x0b\x11', b'\x1a\x1b\t\x03\x0e\x08\x14\t\x0e\x04\n\t\x03\x0e\x08\x12', b'\x1b\x1a\x08\x02\x0f\t\x15\x08\x0f\x05\x0b\x08\x02\x0f\t\x13', b'\x14\x15\x07\r\x00\x06\x1a\x07\x00\n\x04\x07\r\x00\x06\x1c', b'\x15\x14\x06\x0c\x01\x07\x1b\x06\x01\x0b\x05\x06\x0c\x01\x07\x1d', b'\x16\x17\x05\x0f\x02\x04\x18\x05\x02\x08\x06\x05\x0f\x02\x04\x1e', b'\x17\x16\x04\x0e\x03\x05\x19\x04\x03\t\x07\x04\x0e\x03\x05\x1f', b'\x10\x11\x03\t\x04\x02\x1e\x03\x04\x0e\x00\x03\t\x04\x02\x18', b'\x11\x10\x02\x08\x05\x03\x1f\x02\x05\x0f\x01\x02\x08\x05\x03\x19', b'\x12\x13\x01\x0b\x06\x00\x1c\x01\x06\x0c\x02\x01\x0b\x06\x00\x1a', b'\x13\x12\x00\n\x0

In [19]:
# Lots of options, can use frequency analysis to find a good one

import string
from collections import defaultdict

englishLetterFreq = {
    'E': 12.70, 'T': 9.06, 'A': 8.17, 'O': 7.51, 'I': 6.97,
    'N': 6.75,  'S': 6.33, 'H': 6.09, 'R': 5.99, 'D': 4.25,
    'L': 4.03,  'C': 2.78, 'U': 2.76, 'M': 2.41, 'W': 2.36,
    'F': 2.23,  'G': 2.02, 'Y': 1.97, 'P': 1.93, 'B': 1.29,
    'V': 0.98,  'K': 0.77, 'J': 0.15, 'X': 0.15, 'Q': 0.10,
    'Z': 0.07,  ' ': 18.0
}

def freq(s):
    d = defaultdict(int)
    for x in s:
        d[x] += 1

    score = 0
    for c, val in d.items():
        if chr(c) in string.printable:
            if chr(c).capitalize() in englishLetterFreq.keys():
                score += val * englishLetterFreq[chr(c).capitalize()]
    return score


best_candidate = []
for candidate in candidates:
    score = freq(candidate)
    best_candidate.append((score, candidate))
    
best_candidate.sort(key=lambda x : -x[0])
print(best_candidate)

[(100.07000000000001, b'\\]OEHNROHBLOEHNT'), (100.07000000000001, b'|}oehnrohbloehnt'), (85.03000000000002, b'@ASYTRNST^PSYTRH'), (85.03000000000002, b'`asytrnst~psytrh'), (79.94999999999999, b'[ZHBOIUHOEKHBOIS'), (79.94999999999999, b'{zhboiuhoekhbois'), (79.39, b'QPBHEC_BEOABHECY'), (79.39, b'qpbhec\x7fbeoabhecy'), (78.96, b']\\NDIOSNICMNDIOU'), (78.96, b'}|ndiosnicmndiou'), (78.42, b'Z[ICNHTINDJICNHR'), (78.42, b'z{icnhtindjicnhr'), (77.3, b'VWEOBDXEBHFEOBD^'), (77.3, b'vweobdxebhfeobd~'), (74.75, b'GFT^SUITSYWT^SUO'), (74.75, b'gft~suitsywt~suo'), (74.28999999999999, b'WVDNCEYDCIGDNCE_'), (74.28999999999999, b'wvdnceydcigdnce\x7f'), (68.59, b'UTFLAG[FAKEFLAG]'), (68.59, b'utflag{fakeflag}'), (62.59, b'FGU_RTHURXVU_RTN'), (62.59, b'fgu\x7frthurxvu\x7frtn'), (61.959999999999994, b'A@RXUSORU_QRXUSI'), (61.959999999999994, b'a`rxusoru\x7fqrxusi'), (54.0, b"45'- &:' *$'- &<"), (54.0, b"32 *'!= '-# *'!;"), (52.28, b'ML^TY_C^YS]^TY_E'), (52.28, b'ml~ty\x7fc~ys}~ty\x7fe'), (50.449999999999

## Multiple Byte Repeating XOR

If we know chunks of the plaintext we can find chunks of the key

Otherwise do this: https://cryptopals.com/sets/1/challenges/6

## Ciphers

https://www.boxentriq.com/code-breaking/cipher-identifier

If this doesn't work then read this:  
http://www.practicalcryptography.com/cryptanalysis/text-characterisation/identifying-unknown-ciphers/

## Modular Arithmetic

$a \equiv b \mod n \iff n \mid (a - b) \iff a = kn + b$  

Satisfy many properties:  
https://en.wikipedia.org/wiki/Modular_arithmetic#Properties

Useful theorems and things to know:  
[Fermat's Little Theorem](https://en.wikipedia.org/wiki/Fermat%27s_little_theorem)  
[Chinese Remainder Theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem)  
[Euler's Theorem](https://en.wikipedia.org/wiki/Euler%27s_theorem)  
[Euler Totient Function](https://en.wikipedia.org/wiki/Euler%27s_totient_function)  

GCD:
Greatest common divisor of two integers.


## Discrete Logarithm Problem
Normal logarithm problem: $\log_b a$ is a number x such that $b^x = a$  
Discrete logarithm problem: $\log_b a$ is a number k such that $a =b^k \mod n$

Discrete log is a $\textbf{hard}$ problem.

## Diffie-Hellman

https://en.wikipedia.org/wiki/Diffie–Hellman_key_exchange

![alt text](https://upload.wikimedia.org/wikipedia/commons/4/46/Diffie-Hellman_Key_Exchange.svg)

# Rsa

https://en.wikipedia.org/wiki/RSA_(cryptosystem)

Here's an implementation of textbook RSA:

In [28]:
import random
import gmpy2
from Crypto.Util.number import inverse, bytes_to_long, long_to_bytes

def gen_prime():
    base = random.getrandbits(1024)
    off = 0
    while True:
        if gmpy2.is_prime(base + off):
            break
        off += 1
    p = base + off

    return p

class RSA(object):
    def __init__(self):
        pass

    def generate(self, p, q, e=0x10001):
        self.p = p
        self.q = q
        self.N = p * q
        self.e = e
        phi = (p-1) * (q-1)
        self.d = inverse(e, phi)

    def encrypt(self, p):
        return pow(p, self.e, self.N)

    def decrypt(self, c):
        return pow(c, self.d, self.N)
    
print(0x10001)

65537


In [21]:
# Sample RSA Encryption
r = RSA()
p = gen_prime()
q = gen_prime()
r.generate(p, q)

# Convert plaintext message M into an integer m
M = b'utflag{fake_flag}'
m = bytes_to_long(M)
print(f'M = {M.decode()}\nm = {m}')

# Compute the ciphertext
c = r.encrypt(m)
print(f'c = {c}')

# Sample RSA Decryption
dm = r.decrypt(c)
dM = long_to_bytes(dm)
print(f'dm = {dm}\ndM = {dM.decode()}')

M = utflag{fake_flag}
m = 39967759189757816114519692597760878536573
c = 2303477315084725296269689106215377880889198226058104099879367527459906426790798768153536505394668877365739183743790192342077879301930124262851866457576506911870343531642842392396315473440170005260396496558666901576882138248792481196328882656851856257009016778722619364631101874515339558220808419521382632930872042782594136958255791072660258714764336178283166759512686506077230239803728022652232512768922460543099600927706122463475867066042862055422518445448571888037591529362746361578973514466279784102324546665383586210595758516857144066991634065571008883781421491848255364678308073751651382118109696560674046657305
dm = 39967759189757816114519692597760878536573
dM = utflag{fake_flag}


## Tools
Here are some useful websites and tools:  
- CyberChef - https://gchq.github.io/CyberChef/  
- RSACtfTool - https://github.com/Ganapati/RsaCtfTool  
- Wikipedia - https://www.wikipedia.org  
- Identify Cipher - https://www.boxentriq.com/code-breaking/cipher-identifier  
- Sage - http://www.sagemath.org  
- John the Ripper - http://www.openwall.com/john/  
- xortool - https://github.com/hellman/xortool  

For learning more:
- https://github.com/ashutosh1206/Crypton
- https://cryptopals.com

## Will be updating this python notebook in the future with more useful stuff